<h1><center> DS420 - Lab 3: Spark DataFrames

## Goal:

In this lab, you will be working with your new Spark DataFrame skills. You are given ten days of merchandise transactions from a local supermarket in July 2020. Follow along the questoins and complete the tasks below.

__Since our goal in this lab is to practice Spark DataFrame APIs, therefore using SQL queries is not recommended. 5 points will be deducted from your grade for each occurence of using SQL queries.__

## Datasets:

The dataset we going to use consists of three tables coming from the database of a supermarket here in Elizabethtown. The three tables are about products, sales and sellers respectively. The three tables are stored in Parquet file format. You can find the data from a shared directory in HDFS on the ADAL.etown.edu server as described below:

+ `products_parquet: /ds420_shared/products_parquet`
+ `sales_parquet: /ds420_shared/sales_parquet`
+ `sellers_parquet: /ds420_shared/sellers_parquet`

## Q1 (10 points): Start a Spark Session, with the application named as "lab3_xxx", where "xxx" is your last name in lower cases.

#### Import SparkSession module and the SparkSQL functions module with an alias as `F`. 

In [1]:
import findspark

findspark.init('/opt/spark')




#### Create a spark session with name `lab3_xxx`, where xxx is your unique id. 

In [2]:
#   Initialize the Spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('lab3_moynihan').getOrCreate()

import pyspark.sql.functions as F



## Q2 (15 points): Read in all three tables from hdfs into three separate DataFrames. The DataFrame names are already provided.

####  Read in Parquet formated data from the given directories on HDFS. 

Remember since the HDFS is on the same node of the Spark master, you can use `localhost` as the IP address, and use port number `9000` for the HDFS service.

In [3]:
products = spark.read.parquet('hdfs://localhost:9000/ds420_shared/products_parquet')

sales = spark.read.parquet('hdfs://localhost:9000/ds420_shared/sales_parquet')

sellers = spark.read.parquet('hdfs://localhost:9000/ds420_shared/sellers_parquet')


#### Print the first 5 rows of each DataFrame

In [4]:
products.show(5)
sales.show(5)
sellers.show(5)

+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
|         0|   product_0|   22|
|         1|   product_1|   30|
|         2|   product_2|   91|
|         3|   product_3|   37|
|         4|   product_4|  145|
+----------+------------+-----+
only showing top 5 rows

+--------+----------+---------+----------+---------------+--------------------+
|order_id|product_id|seller_id|      date|num_pieces_sold|       bill_raw_text|
+--------+----------+---------+----------+---------------+--------------------+
|       1|         0|        0|2020-07-09|             16|kazugewymyzppnepe...|
|       2|         0|        0|2020-07-10|             86|norxukdtrctjxxtpp...|
|       3|         0|        0|2020-07-01|             48|renkbwhtkyshdcjrh...|
|       4|         0|        0|2020-07-02|             86|whjrxfkmopcgqrdpd...|
|       5|         0|        0|2020-07-02|             22|nyqfeqgdPwmktxzph...|
+--------+----------+---------+----------+-----

#### Print the schema of each DataFrame

In [5]:
products.printSchema()
sales.printSchema()
sellers.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- price: string (nullable = true)

root
 |-- order_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- num_pieces_sold: string (nullable = true)
 |-- bill_raw_text: string (nullable = true)

root
 |-- seller_id: string (nullable = true)
 |-- seller_name: string (nullable = true)
 |-- daily_target: string (nullable = true)



#### Note that the `daily_target` column indicate the "total amount of products sold" daily target for each seller. 

## Q3 (15 points): Basic statistics. Anwer the following questions.

#### Print the total number of rows in each DataFrame

In [6]:
#   Print the number of products
print('Number of products:',   products.count()              )

#   Print the number of orders
print('Number of orders:',        sales.count()           )

#   Print the number of sellers
print('Number of sellers:',        sellers.count()          )


Number of products: 7500000
Number of orders: 2000040
Number of sellers: 10


#### Find how many *unique* products do we have? Rename the resulting column as indicated below.

In [7]:
# Output how many products have been actually sold at least once

sales.select(F.countDistinct('product_id').alias('Unique # of Products')).show()



+--------------------+
|Unique # of Products|
+--------------------+
|               99323|
+--------------------+



## Q4 (15 points): Answer the following questions.

#### Find which product has been sold the most from all orders.

In [8]:
sale = sales.groupBy('product_id').count()
saless = sale.orderBy(F.desc('count')).show(1)

+----------+-------+
|product_id|  count|
+----------+-------+
|         0|1900000|
+----------+-------+
only showing top 1 row



#### Which day has the largest amount of products sold?

In [9]:

sales.groupBy('date').agg({'product_id':'count'}).orderBy(F.desc('count(product_id)')).show(1)



+----------+-----------------+
|      date|count(product_id)|
+----------+-----------------+
|2020-07-06|           201292|
+----------+-----------------+
only showing top 1 row



## Q5 (15 points): If we define the revenue of each product type as "product unit price times number of product sold", calculate the average revenue per order.

#### Step1: join two products and sales tables together using the join() method ([link](https://spark.apache.org/docs/3.2.1/api/python/reference/api/pyspark.sql.DataFrame.join.html)).

In [12]:
# Do the join 
product_sales = products.join(sales, products.product_id == sales.product_id, 'inner')    




In [13]:
product_sales.show(5)

+----------+---------------+-----+--------+----------+---------+----------+---------------+--------------------+
|product_id|   product_name|price|order_id|product_id|seller_id|      date|num_pieces_sold|       bill_raw_text|
+----------+---------------+-----+--------+----------+---------+----------+---------------+--------------------+
|   1034710|product_1034710|  139| 1299770|   1034710|        4|2020-07-07|             98|dvrksgqyuijwuztno...|
|   1036666|product_1036666|  103| 1098542|   1036666|        2|2020-07-09|             90|hzcikhnpnxkkbvflq...|
|   1037840|product_1037840|   48|   99555|   1037840|        3|2020-07-02|             68|ftcvewitmlrcfcgtk...|
|   1043113|product_1043113|   87|   49363|   1043113|        1|2020-07-03|             84|ybaufhttgbmfnitto...|
|   1060548|product_1060548|   35| 1498971|   1060548|        6|2020-07-05|             96|iatuvbdccotrjnboz...|
+----------+---------------+-----+--------+----------+---------+----------+---------------+-----

#### Step 2: create a new column named as 'order_revenue' to the DataFrame `product_sales`  from step 1.

In [22]:
product_sales = product_sales.withColumn('order_revenue', product_sales['num_pieces_sold' ]*product_sales['price'])

In [23]:
product_sales.show(5)

+----------+---------------+-----+--------+----------+---------+----------+---------------+--------------------+-------------+
|product_id|   product_name|price|order_id|product_id|seller_id|      date|num_pieces_sold|       bill_raw_text|order_revenue|
+----------+---------------+-----+--------+----------+---------+----------+---------------+--------------------+-------------+
|   1034710|product_1034710|  139| 1299770|   1034710|        4|2020-07-07|             98|dvrksgqyuijwuztno...|      13622.0|
|   1036666|product_1036666|  103| 1098542|   1036666|        2|2020-07-09|             90|hzcikhnpnxkkbvflq...|       9270.0|
|   1037840|product_1037840|   48|   99555|   1037840|        3|2020-07-02|             68|ftcvewitmlrcfcgtk...|       3264.0|
|   1043113|product_1043113|   87|   49363|   1043113|        1|2020-07-03|             84|ybaufhttgbmfnitto...|       7308.0|
|   1060548|product_1060548|   35| 1498971|   1060548|        6|2020-07-05|             96|iatuvbdccotrjnboz...

#### Step 3: Calculate the average revenue per order. Note the resulting column name has to match the one indicated below. 

In [24]:
product_sales.select(F.avg('order_revenue').alias('Average order revenue')).show()



+---------------------+
|Average order revenue|
+---------------------+
|    1246.602943941121|
+---------------------+



## Q6 (15 points): Answer the following table query questions

#### Find all the products that are in the price range of 50 to 60, inclusive.

In [26]:
products.filter((products['price']>=50 ) & (products['price']<=60)).show()



+----------+------------+-----+
|product_id|product_name|price|
+----------+------------+-----+
|         8|   product_8|   51|
|        10|  product_10|   53|
|        58|  product_58|   54|
|        60|  product_60|   55|
|        62|  product_62|   54|
|        72|  product_72|   60|
|        86|  product_86|   54|
|       102| product_102|   52|
|       103| product_103|   54|
|       147| product_147|   56|
|       168| product_168|   54|
|       177| product_177|   56|
|       178| product_178|   58|
|       205| product_205|   52|
|       207| product_207|   55|
|       211| product_211|   50|
|       229| product_229|   51|
|       258| product_258|   50|
|       264| product_264|   60|
|       276| product_276|   52|
+----------+------------+-----+
only showing top 20 rows



#### Find the average revenue on July 06, 2020. Note the resulting column name has to match the one indicated below. 

In [27]:
avgRev = product_sales.filter(product_sales['date'] == '2020-07-06')
avgRev.select(F.mean('order_revenue').alias('Avg Revenue on 07/06')).show()


+--------------------+
|Avg Revenue on 07/06|
+--------------------+
|  1245.1128311110228|
+--------------------+



## Q7 (15 points): For each seller, what is his/her average per order contribution out of this seller’ `daily_target` in percentage (%)?
Hint: you need to join tables similar to Q5. Creating an additional `per order contribution` column would help on the later step. Feel free to break down to multiple steps. 

In [28]:
# Join the tables

seller_sales = sellers.join(sales, 'seller_id', 'inner')


In [29]:
seller_sales.show(5)

+---------+-----------+------------+--------+----------+----------+---------------+--------------------+
|seller_id|seller_name|daily_target|order_id|product_id|      date|num_pieces_sold|       bill_raw_text|
+---------+-----------+------------+--------+----------+----------+---------------+--------------------+
|        0|   seller_0|      250000|       1|         0|2020-07-09|             16|kazugewymyzppnepe...|
|        0|   seller_0|      250000|       2|         0|2020-07-10|             86|norxukdtrctjxxtpp...|
|        0|   seller_0|      250000|       3|         0|2020-07-01|             48|renkbwhtkyshdcjrh...|
|        0|   seller_0|      250000|       4|         0|2020-07-02|             86|whjrxfkmopcgqrdpd...|
|        0|   seller_0|      250000|       5|         0|2020-07-02|             22|nyqfeqgdPwmktxzph...|
+---------+-----------+------------+--------+----------+----------+---------------+--------------------+
only showing top 5 rows



In [30]:
# Create a new column `contribution`

seller_sales = seller_sales.withColumn('contribution', seller_sales['num_pieces_sold']/seller_sales['daily_target'])


seller_sales.show(5)

+---------+-----------+------------+--------+----------+----------+---------------+--------------------+------------+
|seller_id|seller_name|daily_target|order_id|product_id|      date|num_pieces_sold|       bill_raw_text|contribution|
+---------+-----------+------------+--------+----------+----------+---------------+--------------------+------------+
|        0|   seller_0|      250000|       1|         0|2020-07-09|             16|kazugewymyzppnepe...|      6.4E-5|
|        0|   seller_0|      250000|       2|         0|2020-07-10|             86|norxukdtrctjxxtpp...|     3.44E-4|
|        0|   seller_0|      250000|       3|         0|2020-07-01|             48|renkbwhtkyshdcjrh...|     1.92E-4|
|        0|   seller_0|      250000|       4|         0|2020-07-02|             86|whjrxfkmopcgqrdpd...|     3.44E-4|
|        0|   seller_0|      250000|       5|         0|2020-07-02|             22|nyqfeqgdPwmktxzph...|      8.8E-5|
+---------+-----------+------------+--------+----------+

In [31]:
# Find the final answer

seller_sales.groupBy('seller_name').mean().show()


+-----------+--------------------+
|seller_name|   avg(contribution)|
+-----------+--------------------+
|   seller_1|0.001052995783689955|
|   seller_3|4.044207069691405...|
|   seller_4|6.944752113745378E-4|
|   seller_2|2.617788771371686E-4|
|   seller_7|6.493865185061187E-4|
|   seller_0|2.020522484211978...|
|   seller_5|0.001537585488074...|
|   seller_8|0.001467451290254...|
|   seller_6|4.142882032943009...|
|   seller_9|0.003522315225081...|
+-----------+--------------------+

